# 0-1背包问题
- 有一个背包，它的容量为C(Capacity)。现在有n中不同的物品，编号为0……n-1， 其中每一件物品的重量为$w(i)$，价值为$v(i)$。问可以向这个背包中盛放哪些物品，是的在不超过背包容量的基础上，物品的总价值最大。

## 分析
- 状态定义，$F(n, C)$， 考虑将n个物品放进容量为C的背包，使得价值最大
- 状态转移方程：
    - $F(i,c) = F(i-1, c)$  # 新来一个物品，不放进来
    - $F(i,c) = v(i) + F(i-1, c-w(i))$   # 新来一个物品放进来
    - 以上两种方式取最大
    - $F(i,c) = max(F(i-1, c), v(i) + F(i-1, c-w(i)))$

In [1]:
import time
def countTime(func):
    def wrapper(V, W, idx, C):
        start_time = time.time()
        func(V, W, idx, C)
        count = time.time() - start_time
        print("耗时：{}s".format(count))
    return wrapper

In [2]:
import random
n_goods = 25   # 物品总数
test_C = 100
test_V = [random.randint(1, 8) for i in range(n_goods)]   # 价值数组
test_W = [random.randint(1, 8) for i in range(n_goods)]   # 重量数组
print("价值数组", test_V)
print("重量数组", test_W)

价值数组 [4, 8, 2, 7, 1, 2, 6, 2, 7, 7, 2, 8, 2, 2, 3, 6, 3, 3, 1, 2, 3, 7, 1, 7, 2]
重量数组 [3, 6, 6, 3, 2, 4, 2, 2, 5, 5, 5, 5, 7, 4, 7, 5, 2, 7, 3, 5, 5, 5, 7, 3, 8]


## 1.递归算法

In [3]:
def best_value1(V, W, idx, C):
    if C <= 0 or idx < 0:
        return 0
    res = best_value1(V, W, idx-1, C);
    if W[idx] <= C:
        res = max(res, V[idx] + best_value1(V, W, idx-1, C-W[idx]))
    return res

@countTime
def main(V, W, idx, C):
    print("result:", best_value1(V, W, idx-1, C))
    
main(test_V, test_W, n_goods, test_C)

result: 94
耗时：39.183289766311646s


## 2.记忆化搜索

In [4]:
import numpy as np

In [5]:
def best_value2(V, W, idx, C):
    if C <= 0 or idx < 0:
        return 0
    
    if memo[idx][C] != -1:
        return memo[idx][C]
    
    res = best_value2(V, W, idx-1, C);
    if W[idx] <= C:
        res = max(res, V[idx] + best_value2(V, W, idx-1, C-W[idx]))
    memo[idx][C] = res
    return res

@countTime
def main(V, W, idx, C):
    global memo
    memo = np.array([[-1] * (C+1)] * idx)
    print("result:", best_value2(V, W, idx-1, C))
    
main(test_V, test_W, n_goods, test_C)

result: 94
耗时：0.004984378814697266s


## 3.动态规划

In [6]:
def best_value3(V, W, C):
    assert len(V)==len(W), "value列表和weight列表不一样长"
    n = len(V)
    if n<=0:
        return 0
    memo = np.array([[-1] * (C+1)] * n)
    for j in range(0, C+1):
        memo[0][j] = (V[0] if j>= W[0] else 0)
        
    for i in range(1, n):
        for j in range(0, C+1):
            memo[i][j] = memo[i-1][j]
            if j>=W[i]:
                memo[i][j] = max(memo[i][j], V[i] + memo[i-1][j-W[i]])
    return memo[n-1][C]

@countTime
def main(test_V, test_W, n_goods, test_C):
    print("result:", best_value3(test_V, test_W, test_C))
    
main(test_V, test_W, n_goods, test_C)

result: 94
耗时：0.010972261428833008s


## 4.动态规划的空间优化-1

3中的空间复杂度是$O(N*C)$的，其实可以优化倒$O（2*C）$的

In [7]:
def best_value3(V, W, C):
    assert len(V)==len(W), "value列表和weight列表不一样长"
    n = len(V)
    if n<=0:
        return 0
    memo = np.array([[-1] * (C+1)] * n)
    for j in range(0, C+1):
        memo[0][j] = (V[0] if j>= W[0] else 0)
        
    for i in range(1, n):
        for j in range(0, C+1):
            memo[i%2][j] = memo[(i-1)%2][j]
            if j>=W[i]:
                memo[i%2][j] = max(memo[i%2][j], V[i] + memo[(i-1)%2][j-W[i]])
    return memo[(n-1)%2][C]

@countTime
def main(test_V, test_W, n_goods, test_C):
    print("result:", best_value3(test_V, test_W, test_C))
    
main(test_V, test_W, n_goods, test_C)

result: 94
耗时：0.013961315155029297s


## 5.动态规划的空间优化-2
 
 4中的方法可以进一步优化，空间复杂度上可以优化到真正的$O(C)$，且代码上会更贱简介

In [8]:
def best_value3(V, W, C):
    assert len(V)==len(W), "value列表和weight列表不一样长"
    n = len(V)
    if n<=0:
        return 0
    memo = np.array([-1] * (C+1))
    for j in range(0, C+1):
        memo[j] = (V[0] if j>= W[0] else 0)
        
    for i in range(1, n):
        for j in range(C, -1, -1):
            if j>=W[i]:
                memo[j] = max(memo[j], V[i] + memo[j-W[i]])
    return memo[C]

@countTime
def main(test_V, test_W, n_goods, test_C):
    print("result:", best_value3(test_V, test_W, test_C))
    
main(test_V, test_W, n_goods, test_C)

result: 94
耗时：0.006964921951293945s


## Notice:
二维数组的初始化和数值替换可能有问题，就是因为这个问题，我特么debug了N长时间

In [9]:
n = 5
c = 10
a = [[-1] * (c+1)] * n
a

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]

In [10]:
a[n-1][c] = 9
a

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9]]

In [11]:
b = np.array([[-1] * (c+1)] * n)
b

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [12]:
b[n-1][c] = 9
b

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  9]])